Importing the libraries

In [1]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import numpy as np
import keras

C:\Users\Rens\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Read data file

In [2]:
df2 = pd.read_csv("out_without_nan_mood_target.csv")

Mood back to whole numbers

In [10]:
df=df2
df2['mood_mean_TARGET'] = df2['mood_mean_TARGET'].astype(float)
dfNonNan = df2.dropna(subset = ['mood_mean_TARGET'] )
dfNonNan = pd.concat([dfNonNan,pd.get_dummies(dfNonNan['id'])],axis=1)
df['mood_mean_TARGET'] = df['mood_mean_TARGET'].astype(float).round().astype(str)
df = pd.concat([df,pd.get_dummies(df['id'])],axis=1)

Only select columns with high correlation

In [11]:
corr=dfNonNan.corr()
goodColumns= corr[abs(corr['mood_mean_TARGET'])>0.10]['mood_mean_TARGET'].sort_values()
goodColumns = goodColumns.drop(['mood_mean_TARGET'])
print(goodColumns)

AS14.07                           -0.238830
AS14.12                           -0.171698
AS14.33                           -0.111774
sms_sum_time_5                    -0.106388
weekDay_time_5                    -0.104009
sms_sum_time_4                    -0.100267
appCat.communication_sum_time_5    0.100607
appCat.game_sum_time_3             0.105983
appCat.entertainment_sum_time_3    0.107478
AS14.27                            0.110542
appCat.other_sum_time_5            0.115242
appCat.game_sum_time_5             0.121776
appCat.game_sum_time_4             0.122873
AS14.32                            0.123148
appCat.entertainment_sum_time_4    0.123462
appCat.entertainment_sum_time_5    0.131728
AS14.29                            0.137814
AS14.03                            0.161710
circumplex.valence_mean_time_5     0.211159
circumplex.valence_mean_time_4     0.212472
circumplex.valence_mean_time_3     0.213423
AS14.30                            0.223710
circumplex.valence_mean_time_2  

Split in train and test set

In [5]:
notRelevantAll=df.drop(['id','date'],axis=1).dropna()
goldY= notRelevantAll['mood_mean_TARGET']
relevant = notRelevantAll.drop(['mood_mean_TARGET'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(relevant[goodColumns.keys()], goldY, test_size=0.33, random_state=42)
y_train_cat = keras.utils.to_categorical(y_train.astype(float).astype(int))
y_test_cat = keras.utils.to_categorical(y_test.astype(float).astype(int))

goldY_train = keras.utils.to_categorical(goldY.astype(float).astype(int))

Setup model

In [6]:
model = Sequential()
model.add(Dense(len(X_test.columns), activation='relu', input_shape=(len(X_test.columns),)))
model.add(Dense(8, activation='relu'))
model.add(Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 35)                1260      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 288       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                90        
Total params: 1,638
Trainable params: 1,638
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train_cat,epochs=100, batch_size=1, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
814/814 [==============================] - 2s 2ms/step - loss: 1.4560 - acc: 0.4779
Epoch 2/100
814/814 [==============================] - 1s 1ms/step - loss: 1.1597 - acc: 0.4951
Epoch 3/100
814/814 [==============================] - 1s 1ms/step - loss: 1.1170 - acc: 0.5332
Epoch 4/100
814/814 [==============================] - 1s 1ms/step - loss: 1.0938 - acc: 0.5356
Epoch 5/100
814/814 [==============================] - 1s 1ms/step - loss: 1.0840 - acc: 0.5356
Epoch 6/100
814/814 [==============================] - 1s 1ms/step - loss: 1.0688 - acc: 0.5393
Epoch 7/100
814/814 [==============================] - 1s 1ms/step - loss: 1.0644 - acc: 0.5455
Epoch 8/100
814/814 [==============================] - 1s 1ms/step - loss: 1.0564 - acc: 0.5553
Epoch 9/100
814/814 [==============================] - 1s 952us/step - loss: 1.0439 - acc: 0.5614
Epoch 10/100
814/814 [==============================] - 1s 1ms/step - loss: 1.0460 - a

In [9]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat)

print('Test accuracy:', test_acc)

402/402 [==============================] - 0s 127us/step
Test accuracy: 0.6044776113472174
